In [7]:
#Conexão com o Google Drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
print('TRABALHO AVALIATIVO 2 - ENTREGA 04/06')
print('Aluno: Fábio Rodrigues Garbato')
print('RGM: 13345036')
print('Professora: Osmary Camila Bortoncello Glober (Mary)')
print('')
print('OBJETIVO: Fazer o efeito HoG (COM BIBLIOTECA).')

TRABALHO AVALIATIVO 2 - ENTREGA 04/06
Aluno: Fábio Rodrigues Garbato
RGM: 13345036
Professora: Osmary Camila Bortoncello Glober (Mary)

OBJETIVO: Fazer o efeito HoG (COM BIBLIOTECA).


In [9]:
# Importando Bibliotecas
import cv2
from skimage.feature import hog
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score
import os

In [10]:
def extract_hog_descriptor(image):
    # Converter a imagem para escala de cinza
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # redimensionar para 64x128 pixels
    resized = cv2.resize(gray, (64, 128))

    # Cria o descritor HOG
    hog = cv2.HOGDescriptor()
    
    # Calcula o descritor HOG para a imagem redimensionada
    descriptor = hog.compute(resized)

    return descriptor

image_dir = "/content/drive/MyDrive/Ciência da computação/7° Semestre/Visão Computacional/Trabalho0406/imagens" #Mudar o caminho aqui prof para funcionar no seu ambiente
folders = ['bananas', 'cafes', 'nozes', 'cerejas', 'abacaxis']

descriptors = []
labels = []

for folder in folders:
    folder_path = os.path.join(image_dir, folder)
    image_files = os.listdir(folder_path)
    
    # Verifica se há arquivos de imagem na pasta
    if len(image_files) == 0:
        print(f"A pasta {folder} não contém arquivos de imagem. Pulando para a próxima pasta.")
        continue
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)
        
        # Verifica se a imagem foi lida corretamente
        if image is None:
            print(f"Erro ao ler a imagem {image_path}. Pulando para a próxima imagem.")
            continue
        
        descriptor = extract_hog_descriptor(image)
        descriptors.append(descriptor)


In [11]:
output_file = '/content/drive/MyDrive/Ciência da computação/7° Semestre/Visão Computacional/Trabalho0406/imagens/descritores.txt' #Mudar o caminho aqui prof para funcionar no seu ambiente

# Criação de um conjunto para armazenar os rótulos das pastas já processadas
processed_folders = set()

# Loop pelos descritores
with open(output_file, 'w') as file:
    for folder, descriptor in zip(folders, descriptors):
        label = folder.split('/')[-1]  # Extrai o nome da pasta (rótulo) da imagem
        
        # Verifica se a pasta já foi processada
        if label in processed_folders:
            continue
        
        # Marca a pasta como processada
        processed_folders.add(label)
        
        # Escreve a descrição da imagem no arquivo
        file.write(f"Descritor da imagem 1 da pasta {label}:\n")
        descriptor_str = ','.join(map(str, descriptor))
        file.write(descriptor_str + '\n\n')


In [12]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Rótulos das classes correspondentes aos descritores
for i, folder in enumerate(folders):
    num_images = len(os.listdir(os.path.join(image_dir, folder)))
    labels.extend([folder] * num_images)  

# Divisão dos dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(descriptors, labels, test_size=0.2, random_state=42)

# Definir a grade de valores dos hiperparâmetros a serem testados
param_grid = {'C': [0.1, 1],
              'penalty': ['l2']}

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(LinearSVC(max_iter=10000), param_grid, cv=2)

# Treinar o classificador com validação cruzada
grid_search.fit(X_train, y_train)

# Melhores hiperparâmetros encontrados
best_params = grid_search.best_params_

# Treinar o classificador com os melhores hiperparâmetros
svm = LinearSVC(max_iter=10000, **best_params)
svm.fit(X_train, y_train)

# Predição dos rótulos para os dados de teste
y_pred = svm.predict(X_test)

output_file = '/content/drive/MyDrive/Ciência da computação/7° Semestre/Visão Computacional/Trabalho0406/imagens/rotulos.txt' #Mudar o caminho aqui prof para funcionar no seu ambiente

with open(output_file, 'w') as file:
    for true_label, pred_label in zip(y_test, y_pred):
        file.write(f"Rótulo verdadeiro: {true_label}, Rótulo predito: {pred_label}\n")

# Avaliar o modelo nos dados de teste
print("Relatório de Classificação:\n")
print(classification_report(y_test, y_pred))

Relatório de Classificação:

              precision    recall  f1-score   support

    abacaxis       0.64      0.69      0.67       213
     bananas       0.68      0.60      0.64       251
       cafes       0.45      0.50      0.47       197
     cerejas       0.48      0.41      0.44       203
       nozes       0.41      0.46      0.43       189

    accuracy                           0.54      1053
   macro avg       0.53      0.53      0.53      1053
weighted avg       0.54      0.54      0.54      1053



In [13]:
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

# Cálculo da acurácia do classificador
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy}")
print(f"Acurácia (%): {accuracy * 100:.2f}%")

print()

# Cálculo da precisão do classificador
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision}")
print(f"Precision (%): {precision * 100:.2f}%")

print()

# Cálculo da matriz de confusão
confusion_mat = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(confusion_mat)

print()

# Conclusões gerais
num_classes = len(set(y_test))
print("Conclusões gerais:")
print(" - O classificador apresenta uma acurácia de {:.2f}%.".format(accuracy * 100))
print(" - A precisão média do classificador é de {:.2f}%.".format(precision * 100))
print(" - A matriz de confusão mostra a distribuição dos rótulos verdadeiros e preditos.")
print(" - A matriz de confusão possui dimensões {}x{} para {} classes.".format(
    num_classes, num_classes, num_classes))

Acurácia: 0.5375118708452041
Acurácia (%): 53.75%

Precision: 0.543561453936136
Precision (%): 54.36%

Matriz de Confusão:
[[147   6  31  12  17]
 [ 20 150  21  17  43]
 [ 25   8  98  31  35]
 [ 20  24  46  84  29]
 [ 16  31  24  31  87]]

Conclusões gerais:
 - O classificador apresenta uma acurácia de 53.75%.
 - A precisão média do classificador é de 54.36%.
 - A matriz de confusão mostra a distribuição dos rótulos verdadeiros e preditos.
 - A matriz de confusão possui dimensões 5x5 para 5 classes.
